In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import pre_filter_items
from src.recommenders import MainRecommender

In [2]:
data = pd.read_csv('../data/retail_train.csv')
item_features = pd.read_csv('../data/product.csv')
user_features = pd.read_csv('../data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = pre_filter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [4]:
recommender = MainRecommender(data_train_lvl_1)

In [5]:
recommender.get_als_recommendations(2375, 5)

[899624, 1044078, 871756, 844179, 1106523]

In [6]:
recommender.get_own_recommendations(2375, 5)

[1029743, 1106523, 5569230, 916122, 844179]

In [7]:
recommender.get_similar_items_recommendation(2375, 5)

[1046545, 1044078, 999270, 5568732, 12781666]

In [8]:
recommender.get_similar_users_recommendation(2375, 5)

[1029743, 1106523, 5569230, 916122, 844179]

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?

In [45]:
train_users = data_train_lvl_1['user_id'].unique() # warm start

result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1 = result_lvl_1.loc[result_lvl_1.user_id.isin(train_users)]
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [74]:
result_lvl_1['own_candidates'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, 50))

In [75]:
result_lvl_1['own_recall'] = result_lvl_1.apply(lambda row: recall_at_k(row['own_candidates'], row['actual'], k=50), axis=1)

In [76]:
result_lvl_1['own_recall'].mean()

0.04323684353105271

In [26]:
result_lvl_1['als_candidates'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, 50))

In [28]:
result_lvl_1['als_recall'] = result_lvl_1.apply(lambda row: recall_at_k(row['als_candidates'], row['actual'], k=50), axis=1)

In [29]:
result_lvl_1['als_recall'].mean()

0.04838933079204873

In [32]:
result_lvl_1['sim_item_candidates'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x, 50))

In [33]:
result_lvl_1['sim_item_recall'] = result_lvl_1.apply(lambda row: recall_at_k(row['sim_item_candidates'], row['actual'], k=50), axis=1)

In [34]:
result_lvl_1['sim_item_recall'].mean()

0.033545066881788745

In [62]:
def eval_candidates(df, model, k):
    df = df.copy(deep=True)
    df['candidates_col'] = df['user_id'].apply(lambda x: model(x, k))
    df['recall_col'] = df.apply(lambda row: recall_at_k(row['candidates_col'], row['actual'], k=k), axis=1)
    return df['recall_col'].mean()


In [65]:
result = []
for k in [20, 50, 100, 200, 500]:
    result.append(eval_candidates(result_lvl_1, recommender.get_own_recommendations, k))

In [70]:
result

[0.03033868152652756,
 0.04323684353105271,
 0.06161423470157117,
 0.08943004283810192,
 0.1318966203047918]

In [71]:
result_als = []
for k in [20, 50, 100, 200, 500]:
    result_als.append(eval_candidates(result_lvl_1, recommender.get_als_recommendations, k))

In [72]:
result_als

[0.02964295893294198,
 0.04838933079204873,
 0.06966646065555956,
 0.09790896853094035,
 0.14685862729134996]

### Задание 2

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?